In [ ]:
import pandas as pd
import io
from google.colab import files

uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[filename]))

df.head()

### Adjust headers and columns to be used

In [ ]:
df = pd.read_excel(url, sheet_name='Raw Data', header=(2))
df = df.iloc[:, :4]
df.head()

,Hour,Date/hour start,Solar electricity generation (kWh),Electricity usage (kWh)
0,0.0,2020-01-01 00:00:00,0.0,1.509849
1,1.0,2020-01-01 01:00:00,0.0,1.411859
2,2.0,2020-01-01 02:00:00,0.0,1.023898
3,3.0,2020-01-01 03:00:00,0.0,0.642000
4,4.0,2020-01-01 04:00:00,0.0,0.960000


### Determine data types and statistical summary of numerical columns in our data set

In [ ]:
#Data Types
print(df.dtypes)


Hour                                         float64
Date/hour start                       datetime64[ns]
Solar electricity generation (kWh)           float64
Electricity usage (kWh)                      float64
dtype: object


In [ ]:
#Statistical Summary
df.describe()

,Hour,Date/hour start,Solar electricity generation (kWh),Electricity usage (kWh)
count,8760.000000,8760,8760.000000,8761.000000
mean,11.500000,2020-07-02 07:37:13.972602624,1.116750,12.562411
min,0.000000,2020-01-01 00:00:00,0.000000,-12.624000
25%,5.750000,2020-04-02 05:45:00,0.000000,0.300000
50%,11.500000,2020-07-02 11:30:00,0.024000,0.621000
75%,17.250000,2020-10-01 17:15:00,1.272750,1.686000
max,23.000000,2020-12-31 23:00:00,13.050000,46000.000000
std,6.922582,NaN,2.026098,694.962575


**Data Cleaning**

In [ ]:
!pip install sweetviz

import sweetviz as sv



# Generate a Sweetviz report
report = sv.analyze(df)
# Save the report to an HTML file
report.show_html('electricity_usage_report.html')


from google.colab import files
files.download('electricity_usage_report.html')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 50.6 MB/s eta 0:00:00


                                             |          | [  0%]   00:00 -> (? left)

Report electricity_usage_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# Missing data
null= df.isnull().sum()
print('missing data', null)



In [ ]:
#Split Date/Hour column into two

df['Date'] = df['Date/hour start'].dt.date.
df['Time'] = df['Date/hour start'].dt.strftime('%H:%M:%S')
df.drop(columns=['Date/hour start'], inplace=True)
#Rename the solar and electricity columns for easy recall
df.rename(columns={'Solar electricity generation (kWh)': 'Solar', 'Electricity usage (kWh)': 'Electricity'}, inplace=True)

print(df.columns)

Average solar and electricity use per hour

#### (i)	Carry out checks on the data provided to confirm that the data is complete and fit for use. This should include:

•	creating a graph showing the average solar electricity generation and average electricity usage for each hour in a day, i.e. the average over all amounts for 1 am, and each subsequent hour within a 24-hour period.

•	investigating any significant outliers.

•	making any corrections that are needed.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate hourly averages
hourly_avg = df.groupby('Hour').agg({
    'Solar': 'mean',
    'Electricity': 'mean'
}).reset_index()

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(hourly_avg['Hour'], hourly_avg['Solar'],
         label='Avg Solar Generation', marker='o')
plt.plot(hourly_avg['Hour'], hourly_avg['Electricity'],
         label='Avg Electricity Usage', marker='o')

plt.xlabel('Hour of Day')
plt.ylabel('Average Electricity (kWh)')
plt.title('Average Solar Generation vs Electricity Usage by Hour')
plt.legend()
plt.grid(True)
plt.xticks(range(0, 24))
plt.show()

##### Check for significant outliers

In [ ]:
# Check for outliers using z-score method
import numpy as np
from scipy import stats

# Calculate z-scores for both metrics
z_scores_solar = np.abs(stats.zscore(df['Solar']))
z_scores_usage = np.abs(stats.zscore(df['Electricity']))

# Find outliers (z-score > 3)
outliers_solar = df[z_scores_solar > 3]
outliers_usage = df[z_scores_usage > 3]

print("Solar Generation Outliers:")
print(outliers_solar.sort_values('Solar', ascending=False).head())

print("\
Electricity Usage Outliers:")
print(outliers_usage.sort_values('Electricity', ascending=False).head())

# Create box plots to visualize the distribution and outliers
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.boxplot(df['Solar'])
plt.title('Solar Generation Distribution')
plt.ylabel('kWh')

plt.subplot(1, 2, 2)
plt.boxplot(df['Electricity'])
plt.title('Electricity Usage Distribution')
plt.ylabel('kWh')

plt.tight_layout()
plt.show()



##### Make corrections to our data

In [ ]:
import matplotlib.pyplot as plt

def replace_outliers(df, column):
  df_no_outliers = df.copy()  # Creating a copy of the dataframe
  # Calculate Q1 (25th percentile) and Q3 (75th percentile)
  Q1 = df_no_outliers[column].quantile(0.25)
  Q3 = df_no_outliers[column].quantile(0.75)
  # Calculate the IQR (Interquartile Range)
  IQR = Q3 - Q1
  # Define outlier boundaries
  lower_bound = Q1 - 3 * IQR
  upper_bound = Q3 + 3 * IQR
  # Identify outliers
  outliers = (df_no_outliers[column] < lower_bound) | (df_no_outliers[column] > upper_bound)
  # Calculate the median of the column
  median_value = df_no_outliers[column].median()  # Now df_no_outliers is defined
  # Replace outliers with the median
  df_no_outliers.loc[outliers, column] = median_value
  return df_no_outliers
df_no_outliers= replace_outliers(df, 'Solar')
df_no_outliers= replace_outliers(df_no_outliers, 'Electricity')


In [ ]:
df_no_outliers.describe()

In [ ]:
# Calculate hourly averages
hourly_avg_clean =df_no_outliers.groupby('Hour').agg({
    'Solar': 'mean',
    'Electricity': 'mean'
}).reset_index()

# Create the updated plot
plt.figure(figsize=(12, 6))
plt.plot(hourly_avg['Hour'], hourly_avg['Solar'],
         label='Avg Solar Generation', marker='o')
plt.plot(hourly_avg_clean['Hour'], hourly_avg_clean['Electricity'],
         label='Avg Electricity Usage', marker='o')

plt.xlabel('Hour of Day')
plt.ylabel('Average Electricity (kWh)')
plt.title('Average Solar Generation vs Electricity Usage by Hour (Outliers Removed)')
plt.legend()
plt.grid(True)
plt.xticks(range(0, 24))
plt.show()


# Create box plots to visualize the distribution and outliers
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.boxplot(df_no_outliers['Solar'])
plt.title('Solar Generation Distribution(Outliers Replaced)')
plt.ylabel('kWh')

plt.subplot(1, 2, 2)
plt.boxplot(df_no_outliers['Electricity'])
plt.title('Electricity Usage Distribution(Outliers Replaced)')
plt.ylabel('kWh')

plt.tight_layout()
plt.show()

print("\
Original data:", len(df))
print("Data after replacing outliers:", len(df_no_outliers))
print("\
Updated hourly averages:")
print(hourly_avg_clean)

In [ ]:
import sweetviz as sv
import pandas as pd


# Generate a Sweetviz report
report = sv.analyze(df_no_outliers)
# Save the report to an HTML file
report.show_html('electricity_usage_report_clean1.html')


from google.colab import files
files.download('electricity_usage_report_clean1.html')

### (ii)	Calculate, for each hour in 2020, the amount of electricity that needed to be bought from the electricity provider (measured in kWh and subject to a minimum of zero).


In [ ]:
# Calculate the electricity needed to be bought from the provider
df_no_outliers['Electricity bought(no battery)'] = df_no_outliers.apply(
    lambda row: max(0, row['Electricity'] - row['Solar']), axis=1
)

# Display the first few rows of the updated dataframe
print(df_no_outliers[['Hour', 'Date','Time', 'Electricity bought(no battery)']].head())


### (iii)	Calculate, for each hour in 2020, the excess solar electricity generated over electricity used (measured in kWh and subject to a minimum of zero).


In [ ]:
# Excess solar electricity
df_no_outliers['Excess solar'] = df_no_outliers.apply(
    lambda row: max(0, float(row['Solar']) - float(row['Electricity'])),
    axis=1
)


print("Excess solar calculations for 2020:")
print(df_no_outliers[['Hour', 'Date','Time','Solar','Electricity','Electricity bought(no battery)', 'Excess solar']])

# Calculate total excess solar in 2020
total_excess = df_no_outliers['Excess solar'].sum()
print(f"\
Total excess solar electricity in 2020: {total_excess:.2f} kWh")

### (iv) Model the cumulative battery charge level (measured in kWh) for each hour over 2020, assuming a battery had already been installed.

The battery charge level should:

•	begin at zero at 1 January 2020 00:00.

•	allow for the increase or decrease in charge level depending on the hourly results of parts (ii) and (iii).

•	be subject to the cap on the maximum battery charge level.


In [ ]:
# Ensure the data type for numerical columns is float
battery_capacity = 12.5
battery_charge = 0.0

# Convert relevant columns to float
df_no_outliers['Excess solar'] = df_no_outliers['Excess solar'].astype(float)
df_no_outliers['Electricity bought(no battery)'] = df_no_outliers['Electricity bought(no battery)'].astype(float)
df_no_outliers['Battery charge'] = 0.0

# calculate the battery charge level
for index, row in df_no_outliers.iterrows():
    battery_charge += row['Excess solar'] - row['Electricity bought(no battery)']
    battery_charge = max(0.0, min(battery_charge, battery_capacity))
    df_no_outliers.at[index, 'Battery charge'] = battery_charge

# Plot updated results
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_no_outliers, x='Date', y='Battery charge')
plt.title('Battery Charge Level Throughout 2020')
plt.xlabel('Date')
plt.ylabel('Battery Charge (kWh)')
plt.axhline(y=12.5, color='r', linestyle='--', label='Maximum Capacity')
plt.legend()
plt.show()

print(f"Average battery charge: {df_no_outliers['Battery charge'].mean():.2f} kWh")
print(f"Maximum battery charge reached: {df_no_outliers['Battery charge'].max():.2f} kWh")

### (v) Calculate the amount of electricity for each hour in 2020 that would have been bought from the electricity provider (measured in kWh and subject to a minimum of zero), assuming a battery had already been installed.

In [ ]:
# Electricity bought(battery) installed
df_no_outliers['Electricity bought(battery)'] = 0.0
battery_charge = 0.0

for index, row in df_no_outliers.iterrows():
    # Calculate required electricity
    required = float(row['Electricity'])
    solar_gen = float(row['Solar'])

    # Use solar first
    needed_after_solar = max(0, required - solar_gen)

    # Use battery if available
    used_from_battery = min(battery_charge, needed_after_solar)
    needed_after_battery = needed_after_solar - used_from_battery

    # Update battery charge
    battery_charge -= used_from_battery
    excess_solar = max(0, solar_gen - required)
    battery_charge = min(battery_capacity, battery_charge + excess_solar)

    # Electricity that needs to be bought
    df_no_outliers.at[index, 'Electricity bought(battery)'] = needed_after_battery


print("Electricity bought(battery) installed:")
print(df_no_outliers[['Date', 'Electricity bought(battery)']].head())

total_bought_with_battery = df_no_outliers['Electricity bought(battery)'].sum()
total_bought_without_battery = df_no_outliers['Electricity bought(no battery)'].sum()

print(f"\
Total electricity bought in 2020 with battery: {total_bought_with_battery:.2f} kWh")
print(f"Total electricity bought in 2020 without battery: {total_bought_without_battery:.2f} kWh")
print(f"Reduction in electricity bought: {total_bought_without_battery - total_bought_with_battery:.2f} kWh")

### (V) Calculate the savings over 2020 (in dollars ($), using 1 January 2022 electricity prices and ignoring discounting) from installing a battery compared to using the existing solar panels alone.

In [ ]:
# price of $0.15 per kWh for this calculation as provided in the background information

electricity_price_per_kwh = 0.15

# Total cost with and without battery
total_cost_with_battery = df_no_outliers['Electricity bought(battery)'].sum() * electricity_price_per_kwh
total_cost_without_battery = df_no_outliers['Electricity bought(no battery)'].sum() * electricity_price_per_kwh

# Savings
savings = total_cost_without_battery - total_cost_with_battery

print(f"Total cost with battery: ${total_cost_with_battery:.2f}")
print(f"Total cost without battery: ${total_cost_without_battery:.2f}")
print(f"Savings from installing battery: ${savings:.2f}")

### (vi) Tabulate the data appropriately and then produce a chart to illustrate, on a monthly basis for the calendar year and measured in kWh, the:

•	monthly solar generation.

•	monthly electricity usage.

•	monthly electricity purchased from the electricity provider (no battery).

•	monthly electricity purchased from the electricity provider (with battery).


In [ ]:
# Create monthly aggregations
monthly_data = df_no_outliers.groupby(df_no_outliers['Date'].dt.strftime('%Y-%m')).agg({
    'Solar': 'sum',
    'Electricity': 'sum',
    'Electricity bought(no)': 'sum',
    'Electricity bought(battery)': 'sum'
}).reset_index()

# Rename columns
monthly_data.columns = ['Month', 'Solar Generation', 'Usage', 'Bought (No Battery)', 'Bought (With Battery)']

# Display the tabulated data
print("Monthly Electricity Data:")
print(monthly_data)

# Create visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))
plt.plot(range(12), monthly_data['Solar Generation'], marker='o', label='Solar Generation')
plt.plot(range(12), monthly_data['Usage'], marker='s', label='Usage')
plt.plot(range(12), monthly_data['Bought (No Battery)'], marker='^', label='Bought (No Battery)')
plt.plot(range(12), monthly_data['Bought (With Battery)'], marker='*', label='Bought (With Battery)')

plt.xticks(range(12), [m[5:7] for m in monthly_data['Month']], rotation=0)
plt.xlabel('Month')
plt.ylabel('Electricity (kWh)')
plt.title('Monthly Electricity Patterns in 2020')
plt.legend()
plt.grid(True)
plt.show()